In [6]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [7]:
# set up the timer and buffer of ticks every 10ms
import array, time
from machine import Timer

ntbuff = 40

tindex = 0
tbuff = array.array("I", range(0,ntbuff))
def irqtrigger(timer):
    global tindex
    t1 = time.ticks_ms()
    i = tindex+1
    tbuff[(i%ntbuff)] = t1
    tindex = i
    
timer = Timer(0)
timer.init(period=10, mode=Timer.PERIODIC, callback=irqtrigger)


In [46]:
print(tindex)

85943


In [52]:
# simple version for downloading and plotting
import time
n = 3000
busetriggertimes = True

fout = open("data5.txt", "w")
tindex_old = tindex
for i in range(n):
    if busetriggertimes:
        while tindex_old == tindex:
            time.sleep_ms(1)
        tindex_old += 1
        t = tbuff[(tindex_old%ntbuff)]
    else:
        t = time.ticks_ms()
    fout.write("%08d" % t)
fout.close()

......

In [53]:
%fetchfile --binary data5.txt

Fetched 24000=24000 bytes from data5.txt.


In [55]:
# self-analyze the data between long and short sequences
import time

n = 3000
longdt = 20
busetriggertimes = True

fout = open("data5.txt", "w")
d = [ ]

sumdt = 0
tindex_old = tindex
if busetriggertimes:
    t0 = tbuff[(tindex_old%ntbuff)]
else:
    t0 = time.ticks_ms()
    
i0 = 0
for i in range(n):    
    if busetriggertimes:
        while tindex_old == tindex:
            time.sleep_ms(1)
        tindex_old += 1
        t = tbuff[(tindex_old%ntbuff)]
    else:
        t = time.ticks_ms()
    
    fout.write("%08d" % t)
    dt = t-t0
    if dt > longdt:
        d.append(((i-i0)*8, dt))
        i0 = i
        if len(d) > 20:
            break
    else:
        sumdt += dt
    t0 = t
fout.close()
t = time.ticks_ms()
print("closetime", t-t0)
if i == n-1:
    print("meanshortwrite", sumdt/(n-len(d)))
    print("longwrite_ms", d)
else:
    print(i, n)
    print(d)
    

......closetime 94
meanshortwrite 9.882058
longwrite_ms [(0, 131), (64, 52), (4040, 97), (4096, 99), (4096, 96), (4096, 95), (4096, 98)]


In [56]:
%fetchfile --binary data5.txt

Fetched 24000=24000 bytes from data5.txt.
